# Load file

In [1]:
import numpy as np
import sklearn as sk
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from sklearn import metrics
import gc
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report

import seaborn as sns
from pylab import rcParams
import pickle
from sklearn import metrics
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from pyod.utils.example import visualize
import warnings
import matplotlib.font_manager
warnings.filterwarnings("ignore")

In [2]:
df_path = "C:/Users/Enduser/OneDrive - Asia Pacific University/uni/Y3S2/fyp/Model_trial/btc_trial_dataset2.csv"
dataset_df = pd.read_csv(df_path)

# Drop

In [3]:
drop_cols = ['tx_hash', 'is_malicious', 'all_malicious', 'mean_in_btc', 'mean_out_btc']
dataset_df.drop(columns=drop_cols, inplace=True)

In [4]:
dataset_df.head()

,indegree,outdegree,in_btc,out_btc,total_btc,in_malicious,out_malicious,out_and_tx_malicious
0,4,2,0.478187,0.476987,0.955174,0,0,0
1,3,2,2.019000,2.018500,4.037500,0,0,0
2,1,1,0.180100,0.180100,0.360200,0,0,0
3,1,2,5.879800,5.879300,11.759100,0,0,0
4,4,2,0.495906,0.495406,0.991312,0,0,0


# Spilt

In [5]:
from sklearn.model_selection import train_test_split

# Load dataset
X = dataset_df.drop('out_and_tx_malicious', axis=1)
y = dataset_df['out_and_tx_malicious']

# Split dataset into Train (80%) and Test (20%) ensuring stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Further split Train into Train (90%) and Validation (10%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.10, stratify=y_train, random_state=42
)

print("Data Split Complete:")
print(f"Train Size: {X_train.shape[0]}")
print(f"Validation Size: {X_val.shape[0]}")
print(f"Test Size: {X_test.shape[0]}")


Data Split Complete:
Train Size: 72000
Validation Size: 8000
Test Size: 20000


# log

In [6]:

# Define features for log transformation
log_features = ['indegree', 'outdegree', 'in_btc', 'out_btc', 'total_btc']

# Apply log transformation
for df in [X_train, X_val, X_test]:  
    df[log_features] = np.log1p(df[log_features])  # log1p avoids log(0)

print("Log Transformation Complete")


Log Transformation Complete


In [7]:
# Feature Engineering Function
def add_features(df):
    df['out_malicious_to_total_btc'] = df['out_malicious'] / (df['total_btc'] + 1e-6)
    df['log_total_btc'] = np.log1p(df['total_btc'])
    df['out_malicious_in_btc_interaction'] = df['out_malicious'] * df['in_btc']
    df['net_btc_flow'] = df['in_btc'] - df['out_btc']
    return df

# Apply feature engineering to Train, Validation, and Test sets
X_train_fe = add_features(X_train)
X_val_fe = add_features(X_val)
X_test_fe = add_features(X_test)

# Select the final set of features
selected_features = [
    'in_btc', 'out_btc', 'total_btc', 'out_malicious',
    'out_malicious_to_total_btc', 'log_total_btc',
    'out_malicious_in_btc_interaction', 'net_btc_flow'
]

X_train_final = X_train_fe[selected_features]
X_val_final = X_val_fe[selected_features]
X_test_final = X_test_fe[selected_features]

print("Feature Engineering Complete")


Feature Engineering Complete


In [8]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Apply SMOTE to handle class imbalance
smote = SMOTE(sampling_strategy=0.01, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_final, y_train)

print("Class distribution after SMOTE:")
print(pd.Series(y_train_smote).value_counts())


Class distribution after SMOTE:
out_and_tx_malicious
0    71923
1      719
Name: count, dtype: int64


In [9]:
from sklearn.preprocessing import RobustScaler

# Initialize RobustScaler
scaler = RobustScaler()

# Fit and transform Train set
X_train_scaled = scaler.fit_transform(X_train_smote)

# Transform Validation and Test sets using the same scaler
X_val_scaled = scaler.transform(X_val_final)
X_test_scaled = scaler.transform(X_test_final)

# Convert back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_final.columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val_final.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_final.columns)

print("Scaling Complete")


Scaling Complete


-------------------------------------------

# Model Building - Neural Network

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Set seed for reproducibility
tf.random.set_seed(42)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Enduser\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Enduser\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()

AttributeError: _ARRAY_API not found

- Model building

In [12]:
# Define the Neural Network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Input Layer
    Dropout(0.3),  # Prevent overfitting
    Dense(32, activation='relu'),  # Hidden Layer
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output Layer (binary classification)
])

# Compile the model
nn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['Precision', 'Recall']
)

# Train the model
history = nn_model.fit(
    X_train_scaled, y_train_smote,
    validation_data=(X_test_scaled, y_test),
    epochs=30, batch_size=32,
    verbose=1
)


Epoch 1/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - Precision: 0.1516 - Recall: 0.1840 - loss: 1.4740 - val_Precision: 1.0000 - val_Recall: 0.5455 - val_loss: 0.0067
Epoch 2/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - Precision: 0.6350 - Recall: 0.5177 - loss: 0.3083 - val_Precision: 1.0000 - val_Recall: 0.6364 - val_loss: 0.0165
Epoch 3/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - Precision: 0.8634 - Recall: 0.5176 - loss: 0.0323 - val_Precision: 1.0000 - val_Recall: 0.6364 - val_loss: 0.0118
Epoch 4/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - Precision: 0.9336 - Recall: 0.5189 - loss: 0.0529 - val_Precision: 1.0000 - val_Recall: 0.6364 - val_loss: 0.0086
Epoch 5/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - Precision: 0.9690 - Recall: 0.5234 - loss: 0.0353 - val_Precision: 1.0000 - val_Recall: 0.6364 - val_loss: 0.0081
Epoch 6/30
2271/2271 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - Precision: 0.9860 - Recall: 0.5426 - loss: 0.0211 - val_Precision: 1.0000 - val_Recall: 0.6

- Evaluate

In [14]:
from sklearn.metrics import roc_curve

# Determine the optimal threshold using ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_nn)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Make Predictions
y_pred_nn = (y_pred_proba_nn >= optimal_threshold).astype(int)

# Print Classification Report
print("\nNeural Network Classification Report:")
print(classification_report(y_test, y_pred_nn))



Neural Network Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     19978
           1       0.01      1.00      0.02        22

    accuracy                           0.90     20000
   macro avg       0.51      0.95      0.48     20000
weighted avg       1.00      0.90      0.95     20000



2. 

In [22]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights to handle imbalance
# Compute class weights for imbalanced data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("\n✅ Computed Class Weights:", class_weight_dict)




✅ Computed Class Weights: {0: np.float64(0.5005352946901548), 1: np.float64(467.53246753246754)}


In [23]:
import tensorflow.keras.backend as K

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        return -K.mean(alpha * K.pow(1.0 - pt, gamma) * K.log(pt))
    return loss


In [24]:
# Improved Model with Focal Loss, BatchNorm, Dropout
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled_df.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile Model with Focal Loss
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=focal_loss(),
    metrics=['Precision', 'Recall', 'AUC']
)

# Print Model Summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 256)            │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,313 (177.00 KB)

 Trainable params: 44,417 (173.50 KB)

 Non-trainable params: 896 (3.50 KB)

In [26]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Reduce learning rate when loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

# Train Model
history = model.fit(
    X_train_smote, y_train_smote, 
    validation_data=(X_val_scaled_df, y_val),
    epochs=50,
    batch_size=64, 
    class_weight=class_weight_dict,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


Epoch 1/50
1136/1136 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - AUC: 0.7859 - Precision: 0.0535 - Recall: 0.6786 - loss: 0.1732 - val_AUC: 0.6780 - val_Precision: 0.0080 - val_Recall: 0.5556 - val_loss: 0.2242 - learning_rate: 0.0010
Epoch 2/50
1136/1136 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - AUC: 0.9245 - Precision: 0.4135 - Recall: 0.7132 - loss: 0.0217 - val_AUC: 0.6541 - val_Precision: 0.0091 - val_Recall: 0.5556 - val_loss: 0.2104 - learning_rate: 0.0010
Epoch 3/50
1136/1136 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - AUC: 0.9695 - Precision: 0.5180 - Recall: 0.7047 - loss: 0.0140 - val_AUC: 0.8931 - val_Precision: 0.0177 - val_Recall: 0.5556 - val_loss: 0.0931 - learning_rate: 0.0010
Epoch 4/50
1136/1136 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - AUC: 0.9741 - Precision: 0.5656 - Recall: 0.7114 - loss: 0.0134 - val_AUC: 0.8771 - val_Precision: 0.0128 - val_Recall: 0.5556 - val_loss: 0.1314 - learning_rate: 0.0010
Epoch 5/50
1136/1136 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - AUC: 0.9818 - Precision: 0.6184 - Reca

In [27]:
# Predict Probabilities
y_scores_nn = model.predict(X_test_scaled_df)

# Find best threshold from PR curve
precision, recall, thresholds = precision_recall_curve(y_test, y_scores_nn)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
best_threshold = thresholds[np.argmax(f1_scores)]
print("\n✅ Best Threshold:", best_threshold)

# Convert Probabilities to Binary Predictions
y_pred_nn = (y_scores_nn >= best_threshold).astype(int)

# Classification Report
print("\n📊 Improved Neural Network Results:")
print(classification_report(y_test, y_pred_nn, zero_division=0))


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

✅ Best Threshold: 0.9998209

📊 Improved Neural Network Results:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     19978
           1       0.03      0.59      0.06        22

    accuracy                           0.98     20000
   macro avg       0.52      0.79      0.52     20000
weighted avg       1.00      0.98      0.99     20000

